# Coder Civo Deployment

[Coder](https://coder.com) is a cloud based IDE that allows you to develop in the cloud.  This deployment will create deploy the Coder application to a Civo Kubernetes cluster using terraform and helm. Once up and running you will be able to access the Coder application and start your cloud based coding adventure!

As part of this deployment, we'll be deploying:

- Civo Managed Database
- Civo Managed Kubernetes Cluster (based on the security focused Talos OS)
- Coder (https://coder.com/docs/v2/latest/install/kubernetes)
- Cert Manager (https://cert-manager.io/docs/installation/helm/)
- External DNS

# Civo Account

You can sign up for a civo account at www.civo.com, where new users will get a free $250 credit for a month. That is more than enough for this demo and for trying out the other Civo services.

Once signed in, you will need your API key from https://dashboard.civo.com/security.

# Terraform

[Terraform](https://www.terraform.io/) is a tool for building, changing, and versioning infrastructure safely and efficiently. We'll be using this to deploy the various Civo resources:


In [ ]:
%%bash
sudo apt-get update && sudo apt-get install -y gnupg software-properties-common
wget -O- https://apt.releases.hashicorp.com/gpg | gpg --dearmor | sudo tee /usr/share/keyrings/hashicorp-archive-keyring.gpg
gpg --no-default-keyring --keyring /usr/share/keyrings/hashicorp-archive-keyring.gpg --fingerprint
echo "deb [signed-by=/usr/share/keyrings/hashicorp-archive-keyring.gpg] \
https://apt.releases.hashicorp.com $(lsb_release -cs) main" | \
sudo tee /etc/apt/sources.list.d/hashicorp.list
sudo apt update
sudo apt-get install terraform

# Terraform Variables

Create a file called `terraform.tfvars` and add the following variables:

```hcl
civo_token = "YOUR_API_KEY"
domain = "YOUR_DOMAIN"
```

# Provision Infra

In [ ]:
%%bash
terraform init -upgrade

Create a network for all resources to be deployed within.

In [ ]:
%%bash
terraform apply -target civo_network.network -auto-approve

Next create the managed database and it's firewall.

In [ ]:
%%bash
terraform apply -target civo_database.coder_database -target civo_firewall.database_firewall -auto-approve

Now that the database has been created, let's create a database and the user for coder to use.

In [ ]:
%%bash
terraform apply \
    -target random_password.password \
    -target postgresql_database.coder-database \
    -target postgresql_role.coder \
    -target postgresql_grant.coder \
    -auto-approve

Now the database is set up, we can create the k8s cluster that coder will run on.

In [ ]:
%%bash
terraform apply -auto-approve \
    -target civo_firewall.kube-firewall \
    -target civo_kubernetes_cluster.cluster 

For local debug, we can output the kubeconfig to a file:

In [ ]:
%%bash
terraform apply -auto-approve \
    -target local_file.kubeconfig 

and we can list the nodes in the cluster:

In [ ]:
%%bash
KUBECONFIG=./kubeconfig kubectl get nodes

# Cluster Applications

## Cert Manager

We're installing Cert Manager from helm and installing the CRDs using the helm chart. This shoould not be used for a production deployment when the CRDs should be installed separately.

In [ ]:
%%bash
terraform apply -auto-approve \
    -target helm_release.cert-manager \
    -target kubernetes_manifest.letsencrypt-prod

# Nginx Ingress Controller

This controls traffic ingressing into the cluster. It uses a the `nginx-ingress` helm chart, and a custom annotation of the dedicated CivoFirewall to controll access to applications. This allows us to lock down access at a firewall level, however we could leave this open and haev Nginx control access. To allow for this, we set up `proxy_pass` so that the correct source IP is passed through from the original client.

While we can use External DNS to set a dynamically changing IP address, we may want to ensure a static IP for any future upgrades. As such, we use a Civo Reserved IP address for the ingress to use.


In [ ]:
%%bash
terraform apply -auto-approve \
    -target helm_release.ingress-nginx \
    -target civo_firewall.ingress-nginx \
    -target civo_reserved_ip.ingress-nginx

# External DNS

External DNS will look for ingress resources and create DNS records for them. We'll be using the `external-dns` helm chart to do this, with a domain pre-registered within Civo.

In [ ]:
%%bash
terraform apply -auto-approve \
    -target helm_release.external-dns \
    -target kubernetes_namespace.external-dns \
    -target kubernetes_secret.external-dns-secrets

## Coder

We're installing coder from the upstream helm chart. With the following customisations via the help values file:

- `coder.service.type = ClusterIP` as we're using the nginx ingress controller to expose the service
- `coder.env[].name = CODER_PG_CONNECTION_URL` and `valueFrom` to set the secret to read the DB creds from.
- `coder.ingress.enabled = true` to expose the service via the ingress controller
- `coder.ingress.host` is set to coder.`<domain>` to expose the service via the ingress controller

In [ ]:
%%bash
terraform apply -auto-approve \
    -target kubernetes_namespace.coder \
    -target kubernetes_secret.coder_db \
    -target helm_release.coder